## RSA - The Project

Public key encryption allows people to send and receive information in public with little chance anyone who intercepts that information will be able to decipher it. This is done through the use of 'keys' which are used to encode and decode the information being sent. One type of encryption is RSA encryption (an acronym based on the last names of those who developed the public version of it: Ron Rivest, Adi Shamir, and Leonard Adleman), which uses two public keys, known as n and e, and a private key, known as d, to transmit encrypted information. Using the public and private keys, senders and receivers are able to encode and decode this information, while the encrypted information will be hard for anyone who intercepts it to unlock.

What I have learned while doing this project is that cracking RSA encryption is not as easy as it seems, at least from a brute-force standpoint. Up until a certain point you will be able to factor numbers rather easily. But once you cross the threshold into larger numbers, the amount of time needed to factor a number grows very, very quickly.

I also learned to make sure to utilize all functions written in order to execute the program. At one point I was brute force encoding and decoding. It worked fine for smaller numbers, but the it started to lag. This was when I realized I was not even using my FME function at all. (This function will be explained in detail below.) Note to self: if you write code to do something, make sure you actually call that code! Now on to the project itself.

What follows is code in Python that will allow us to fully implement a basic version of RSA. Before each block of code will be a brief paragraph explaining what that section of code does, and the code itself will have comments within it to help the reader understand how it works.

This first block will be used in the extra section. The time library allows the coder to call functions related to time. For our purposes, we will use it to see how long a process takes to run.

In [1]:
#We'll use this later on in the extra section!!!!!
import random
import math
import time

The function Convert_Text will take a string of words, take each separate character in each word, including spaces and punctuation, convert each character to its ASCII value, and return a list of those values. This will be used as part of the process that encodes messages.

In [2]:
def Convert_Text(_string):
    """
    This function takes in a string and outputs the corresponding
    standard list of integers (ascii) for each letter in the word hello.
    """
    integer_list = []                   #stores conversion of characters to ascii values
    for char in _string:
    #iterates through each character in the string and converts it to ascii value
        integer_list.append(ord(char))
    return integer_list

The function Convert_Num will take a list of numbers (ASCII values) and convert them into their respective characters, generating a string. This will be used as part of the process that decodes messages.

In [3]:
def Convert_Num(_list):
    """
    This function takes in a list of integers and outputs the corresponding string (ascii).
    """
    _string = ''                        #empty string that will store converted numbers
    for i in _list:
    #iterates through numbers in list and converts them to appropriate character
        _string += chr(i)
    return _string

The Convert_Binary_String function takes an integer and converts it into its binary expansion. For example, the number 4 in binary is 0100. For more on binary, see here: https://en.wikipedia.org/wiki/Binary_number.

In [4]:
def Convert_Binary_String(_int):
    """
    Converts an integer to a string of its binary expansion.
    """
    bits = bin(int)[2:]                 #converts integer value to binary value
    return bits

Now that you're done with the basic toolset we'll move on to the first tool set which is actually involved in the RSA system.

FME stands for fast modular exponentiation. This function calculates the remainder when a number raised to a certain power is modular divided by another number. It is necessary to calculate this remainder by means other than brute force mathematics, because exponents in RSA are very large. This makes the time necessary to perform the calculations very time intensive if done is a manner of (x ^ i) % y, which would result in an inefficient means of encryption. Further explanation of the benefits of FME are in Section 6 below.

In [5]:
def FME(b, n, m):
    """
    Fast modular exponentiation function, taking as inputs:
    a = integer
    n = power to which a is being raised
    b = modulus dividing a
    Function calculates the remainder when a raised to the power n
    is modular divided by b.
    """
    result = 1                               #answer that will be returned
    square = b                               #a will be repeatedly squared, mod b
    while(n>0):
    #while loop that continues as long as n, the exponent, is greater than 0
    #loop converts n to binary
        k = n%2                              #extracts least significant bit
        if k == 1:                           #when k = 1, the loop is in its final iteration
            result = (result * square) % m
        square = (square * square) % m       #calculation that continues until k = 1
        n = n // 2                           #finds next largest binary exponent to continue loop
    return result                            #the answer to a^n mod b

The Euclidean_Algorithm function calculates the greatest common divisor (gcd) of two numbers. This function will be used in creating the public and private keys necessary for encryption and decryption.

In [6]:
def Euclidean_Algorithm(a,b):
    """
    finds the greatest common divisor of a and b
    """
    while a != 0:
    #while loop to find gcd; will iterate until b%a == 0, at which point gcd is found
        gcd = a
        a = b%a
        b = gcd
    return b                                 #gcd

The Find_Public_Key_e function will take two relatively prime numbers, p and q, and use them to calculate the public key e. This is accomplished by calling the Euclidean_Algorithm function above.

In [7]:
def Find_Public_Key_e(p, q):
    """
    Takes 2 primes p and q, uses the Euclidean Algorithm to find their gcd.
    Returns public key: n and public key: e. 
    """
    n = p*q                                  #public key n
    phi_n = (p-1)*(q-1)                      #phi-n, needed to compute e
    e = 0
    for e in range (2, phi_n):
    #loop to find e such that e is relatively prime to (p - 1) (q - 1) and not equal to p or q.
        if(Euclidean_Algorithm(e, phi_n)) == 1 and Euclidean_Algorithm(e, phi_n) != p and Euclidean_Algorithm(e, phi_n) != q:
            return n, e
    
    

The Find_Private_Key_d function takes three inputs, p and q from before, and the public key e that we found in the function above. Using these inputs and the Extended Euclidean Algorithm, this function returns the private key d. The Extended Euclidean Algorithm is used to find the modular inverse of e, which is the key d. After finding both e and d, all the tools needed to encrypt and decrypt messages will have been calculated.

In [8]:
def Find_Private_Key_d(e, p, q):
    """
    Finds the decryption exponent d such that d is the modular inverse of e. 
    Uses the Extended Euclidean Algorithm (explained below) to find e.
    
    Returns d: the decryption component.
    """
    (s1, t1) = (1,0)
    (s2,t2) = (0,1)                                #shows how current values of m,n written as linear combination of original values
    phi_n = (p-1)*(q-1)                            #calulates phi-n, which is needed to find modular inverse of e
    p, q = e, phi_n                          

    while q > 0:
    #while loop that executes until the value of n is no longer greater than 0
    #finds GCD and BC using the Extended Euclidean Algorithm
    #continuousy updates the values of m and n in order to find GCD and BC
    #mods final value by phi-n to obtain result and ensure result is positive
        k = p%q
        quo = p//q
        p=q
        q=k
        (s1h, t1h) = (s2, t2)                       #s1h = s1hat, etc, from Sriram's video, necessary to get correct
        (s2h, t2h) = (s1 - quo*s2, t1 - quo*t2)     #values associated with (s1, t1) and (s2, t2) in order
        (s1, t1) = (s1h, t1h)                       #to find GCD and BC; (s1 - q*s2, t1 - q*t2) --> computes
        (s2, t2) = (s2h, t2h)                       #new value for (s2, t2) using quotient from above
    return s1 % phi_n

The Encode function takes the message a user wants to send, along with n (the product of p * q), and the key e found above to encrypt the message. The message will be encrypted using the Convert_Text function described above, as well as the FME function to return a list of numbers representing the characters that have been encrypted. An example of one of these lists can be seen in section 4 below.

In [9]:
def Encode(n, e, message):
    """
    Takes a string of characters, uses the function Convert_Text to convert
    characters into a list of numbers.
    
    Encodes each of those numbers using n and e and returns the encoded cipher_text.
    """
    cipher_text=[]
    num_list = Convert_Text(message)
    for item in num_list:
    #iterates through list of characters and converts them to numbers
        item = FME(item, e, n)
        cipher_text.append(item)
    return cipher_text

The Decode function takes the message received, written as a series of encoded numbers, and decodes it using the value n, the private key d, the FME function, and the Convert_Num functions described above. The result will be the actual message that was written before being encrypted by the sender.

In [10]:
def Decode(n, d, cipher_text):
    """
    Takes cipher_text, a list of integers, and decrypts each of those integers using n and d.
    Uses the function Convert_Num, that converts the integers to a string, to return the original message as a string. 
    """
    new_list = []
    for item in cipher_text:
    #iterates through a list of numbers and converts them to characters
        item = FME(item, d, n)
        new_list.append(item)
    return(Convert_Num(new_list))

Now that we know what RSA is, and we've gone through the code that will make it work, here is an example of the entire process in action.

Step 1: pick p and q. Remember, these numbers must be relatively prime. In order to start small, we'll go with 59 and 83.

In [11]:
p, q = 59, 83

Next, test to make sure the numbers are relatively prime using the Euclidean_Algorithm function. If the result is 1, the numbers are relatively prime.

In [12]:
print(Euclidean_Algorithm(p, q))

1


Since the numbers are relatively prime, the next step is to find n and our public key e using the Find_Public_Key_e function.

In [13]:
n, e = Find_Public_Key_e(p, q)
print("n =", n, ",", "e =", e)

n = 4897 , e = 3


Now that we know the public keys n and e, we can use these values and the Encode function to encode our message.

In [14]:
message = "Here is the key to the RSA project: https://bit.ly/3bFItZU"
encoded_message = Encode(n, e, message)

The Encode function goes through each character in the string, message, converts it to its ASCII equivalent, appends those items to a list, and then returns those ASCII values as a list. The following step is not necessary in encryption, but is useful to see the output of the Encode function.

In [15]:
print(encoded_message)

[1076, 1931, 2650, 1931, 3386, 1933, 2805, 3386, 3650, 3451, 1931, 3386, 793, 1931, 3744, 3386, 3650, 1368, 3386, 3650, 3451, 1931, 3386, 2904, 3735, 393, 3386, 4386, 2650, 1368, 1045, 1931, 693, 3650, 4129, 3386, 3451, 3650, 3650, 4386, 2805, 4129, 986, 986, 968, 1933, 3650, 4293, 1183, 3744, 986, 432, 968, 210, 2154, 3650, 4244, 2000]


Now we've created our encoded message and it is ready to be sent! However, in order for the sender to decode the message, the sender needs to find their private key, d. We can accomplish this using the values of e, p, q, and the Find_Private_Key_d function.

In [16]:
d = Find_Private_Key_d(e, p, q)
print(d)

3171


We now have everything we need to decode our message! This will be accomplished using our Decode function. In order to decode, we need to input the values of n and d, and the encoded message.

In [17]:
decoded_message = Decode(n, d, encoded_message)

Now we can print the decoded message!

In [18]:
print(decoded_message)

Here is the key to the RSA project: https://bit.ly/3bFItZU


This was the original message, so we know that all of our code worked!
(If you click the link, you will be Rickrolled! See https://en.wikipedia.org/wiki/Rickrolling for what Rickrolling is if you don't know.)

But how does it work in reverse? What if we start with an encoded message, and want to encode a reply? Simple. First, take an encoded message. We'll take one posted by a fellow student in Piazza.

In [19]:
encoded_piazza_message = [25463, 2136, 27427, 32768, 34537, 13902, 40606, 2136, 25903, 32768, 23033, 38767, 28099, 32768, 25903, 23587, 7692, 7692, 33653, 28099, 32768, 40606, 2136, 3352, 32768, 24397, 38767, 32768, 15178, 33653, 13902, 33653, 27598, 28099, 40606, 24397, 33653, 32768, 24397, 3635, 33653, 32768, 33653, 2136, 3352, 32768, 38767, 23033, 32768, 24397, 3635, 36396, 25903, 32768, 25903, 33653, 7692, 33653, 25903, 24397, 33653, 28099, 885, 32768, 15274, 32768, 28099, 33653, 40606, 13902, 13902, 27427, 32768, 24079, 36396, 25903, 3635, 32768, 24397, 38767, 32768, 24397, 28099, 40606, 23479, 33653, 13902, 32768, 36396, 2136, 24397, 33653, 28099, 2136, 40606, 24397, 36396, 38767, 2136, 40606, 13902, 13902, 27427, 32768, 40606, 13025, 40606, 36396, 2136, 35937]

In Piazza, all values needed are given, so we can set the values accordingly.

In [20]:
n, e, d = 41527, 3, 27387

In order to decode the message, we need use the decode function we used before.

In [21]:
decoded_piazza_message = Decode(n, d, encoded_piazza_message)
print(decoded_piazza_message)

Any plans for summer and to celebrate the end of this semester? I really wish to travel internationally again!


Now we have selected two relatively prime numbers and used the functions we have written and these numbers to calculate n, find our public key e, our private key d, to encode our own message, decode it, and to decode someone else's message from Piazza. Although commercial-level RSA uses much bigger prime numbers, the fundamental process of encryption is the same as the basic version we have gone through in this project.

If you create a custom main function or a script to read codes as your CUSTOM FEATURE, you may include it here with a large heading CUSTOM FEATURE:

For all Piazza exchanges we are given n, e, d, and a message to decode. These will be shown in consecutive code blocks below, using the following format:
- Author of Piazza post
- Keys needed to decrypt message and reply
- Original encoded message
- Decoded message
- Reply before encoding
- Reply after encoding

Example 1:

In [22]:
#Author: Alex Dowdell
n, e, d = 9797, 7, 2743
alex_message = [4974, 9305, 6336, 2222, 3675, 3179, 2222, 3675, 6336, 5432, 9305, 3675, 5432, 1908, 1908, 3675, 8785, 7721, 5432, 9710, 2222, 1908, 3675, 5432, 1643, 5432, 4668, 9305, 6496, 3675, 3179, 1177, 2222, 7721, 2222, 3675, 3179, 7969, 8564, 1908, 6261, 3675, 7911, 7969, 8564, 3675, 1908, 4668, 2792, 2222, 3675, 8785, 7969, 3675, 1643, 7969, 3675, 9710, 4668, 1622, 4668, 8785, 3675, 8785, 1177, 2222, 3675, 493, 7969, 1622, 8785, 4604]
alex_decode = Decode(n, d, alex_message)
print(alex_decode)

Once we can all travel again, where would you like to go visit the most?


In [23]:
russ_reply = "Either to the beach or to the mountains!"
russ_encode = Encode(n, e, russ_reply)
print(russ_encode)                      #printed just to show what message looks like in encoded form

[3900, 4668, 8785, 1177, 2222, 7721, 3675, 8785, 7969, 3675, 8785, 1177, 2222, 3675, 4075, 2222, 5432, 6336, 1177, 3675, 7969, 7721, 3675, 8785, 7969, 3675, 8785, 1177, 2222, 3675, 493, 7969, 8564, 9305, 8785, 5432, 4668, 9305, 1622, 3833]


Example 2:

In [24]:
#Author: Connor Monk
n, e, d = 15707, 5, 12365
connor_message = [1139, 12480, 2002, 4280, 12763, 10548, 4439, 12763, 4280, 14240, 4439, 4280, 2600, 12763, 9217, 12794, 15705, 4280, 12178, 2002, 2600, 4280, 7769, 5770, 7903, 7903, 12763, 2600, 4280, 1448, 12763, 9217, 469, 12480, 12763, 2600, 5455]
connor_decode = Decode(n, d, connor_message)
print(connor_decode)

Who else is ready for Summer weather?


In [25]:
russ_reply = "I'm not quite ready for summer, but I am definitely ready for spring!"
russ_encode = Encode(n, e, russ_reply)
print(russ_encode)                     #printed just to show what message looks like in encoded form

[14612, 3191, 7903, 4280, 6085, 2002, 469, 4280, 9379, 5770, 14240, 469, 12763, 4280, 2600, 12763, 9217, 12794, 15705, 4280, 12178, 2002, 2600, 4280, 4439, 5770, 7903, 7903, 12763, 2600, 8431, 4280, 2245, 5770, 469, 4280, 14612, 4280, 9217, 7903, 4280, 12794, 12763, 12178, 14240, 6085, 14240, 469, 12763, 10548, 15705, 4280, 2600, 12763, 9217, 12794, 15705, 4280, 12178, 2002, 2600, 4280, 4439, 5762, 2600, 14240, 6085, 909, 9256]


Example 3:

In [26]:
#Author: Emily Carpenter
n, e, d = 7663, 959, 1343
emily_message = [587, 5298, 6099, 4780, 628, 5990, 7581, 3802, 3101, 1071, 3101, 5022, 4760, 4405, 6373, 3101, 3531, 4760, 4405, 3101, 628, 3312, 5990, 3101, 4116, 1071, 7569, 3802, 3101, 1071, 3101, 2037, 2417, 3101, 7581, 4760, 628, 3101, 2037, 3101, 4780, 4572, 1742, 3802, 3101, 1071, 3101, 1218, 1489, 4780, 628, 3101, 4405, 5990, 2037, 259, 3101, 2620, 4760, 4760, 6373, 4780, 2281, 587, 3101, 3133, 3312, 2037, 628, 587, 4780, 3101, 1742, 4760, 1489, 4405, 3101, 3531, 2037, 1298, 4760, 4405, 6099, 628, 5990, 3101, 6099, 7581, 628, 5990, 4224, 4224, 6099, 372, 5990, 7581, 2183, 5990, 3101, 2417, 4760, 1298, 6099, 5990, 7158, 3101, 3167, 6099, 7581, 5990, 3101, 6099, 4780, 3101, 4055, 3101, 5054, 2037, 1742, 4780, 3101, 4760, 3531, 3101, 628, 3312, 5990, 3101, 4116, 4760, 7581, 259, 4760, 4405, 3802]
emily_decode = Decode(n, d, emily_message)
print(emily_decode)

'Listen. I work for the CIA. I am not a spy. I just read books!' What's your favorite intelligence movie? Mine is 3 Days of the Condor.


In [27]:
russ_reply = "How about one comedy and one drama, both worth watching: Top Secret! and Bridge of Spies."
russ_encode = Encode(n, e, russ_reply)
print(russ_encode)                     #printed just to show what message looks like in encoded form

[3038, 4760, 5022, 3101, 2037, 2620, 4760, 1489, 628, 3101, 4760, 7581, 5990, 3101, 2183, 4760, 2417, 5990, 259, 1742, 3101, 2037, 7581, 259, 3101, 4760, 7581, 5990, 3101, 259, 4405, 2037, 2417, 2037, 2705, 3101, 2620, 4760, 628, 3312, 3101, 5022, 4760, 4405, 628, 3312, 3101, 5022, 2037, 628, 2183, 3312, 6099, 7581, 372, 2032, 3101, 2895, 4760, 4572, 3101, 4552, 5990, 2183, 4405, 5990, 628, 2281, 3101, 2037, 7581, 259, 3101, 4099, 4405, 6099, 259, 372, 5990, 3101, 4760, 3531, 3101, 4552, 4572, 6099, 5990, 4780, 3802]


What was challenging? For the exchanging part it was pretty straight forward. If you know the keys the only thing you need to do is make sure the functions work. Mine worked, so it was just a matter of encoding and decoding. (I did run into one problem, but that will be addressed in a later question.)

Who helped you with your project? I did the project on my own. Although I guess in order to get full credit I had to decode messages on Piazza and include those messages here, so in that sense the people whose messages I decoded (Alex, Connor, and Emily) helped me.

Which resources were most helpful? Sriram's videos 1000%. There is no way I could have done this without those videos. His algorithms and his explanations of how they worked were what made the coding part of this as straightforward as it was.

What was your Best Mistake? When I was first encoding and decoding messages, I hard coded the FME part (I did not call the function). This worked fine when the values were small, but then I went to decode a message and 45 minutes later it was still decoding. I thought, "There has to be a faster way to do this!" Then I looked at the code to see what I was doing, and realized it was the slow form of FME. It was then I realized that I had not used FME up until that point, and that calling FME might be a good idea! After that, everything worked just fine. And a lot quicker!

This ties back in to my "Best Mistake." I tried to just brute force encoding and decoding the messages with the following code:

In [28]:
def pseudo_encode(n, e, message):                           #we won't call this function at all, but I want to define it like we will
    cipher_text=[]
    num_list = Convert_Text(message)
    for item in num_list:
    #iterates through list of characters and converts them to numbers
        item = (item**e) % n
        cipher_text.append(item)
    return cipher_text

def psuedo_decode(n, d, cipher_text):
    new_list = []
    for item in cipher_text:
        item = (item**d) % n
        new_list.append(item)
    return(Convert_Num(new_list))

This worked fine for smaller values, but once the values grew to around five or six digits, my computer took forever to process them. Finally, after 45 minutes of trying, my computer still hadn't decoded one message. That was when I finally realized I was doing something wrong and that I hadn't implemented FME. For a more detailed explanation of what I tested, I'll go to my answer from the Number Theory MW:

After reading the posts on Piazza I started with an exponent in the 10,000s and FakeFME still finished rather quickly. Then I read Beth’s post on trying larger exponents. So I went all in and tried 5 ^ 20481091823 % 17. After 45 minutes FakeFME still had not finished. I then ran the same numbers using FME. I got the result (7) in 0.3125 seconds. I ran it a second time and got the same result. I know I went to the extreme first, and I knew it would be faster, but I didn’t think it would finish instantly.

For more traditional testing, I gradually increased the exponents and got the following runtimes:

exponent: 1000, 10000, 100000, 1000000, 10000000, 10000000
FakeFME: .003125, .003125, .003125, .21875, 5.65625, 215.453125
fme: .003125, .003125, .003125, .003125, .003125, .003125

Somewhere between the powers of 100000 and 1000000 fme becomes faster than FakeFME. When the exponent hits 10000000 fme is over 1800 times faster. But when the exponent increases to 100000000, fme is 68945 times faster than FakeFME!!! What’s even more amazing is the exponent I mentioned in the beginning of this, 20481091823 was still calculated using fme in the same amount of time it took to calculate a four-digit exponent. I can’t quantify how much faster fme was because, after 45 minutes, I chose not to let FakeFME finish.


## CODE BREAKING ##

Begin by coding the basic brute force factorization algorithm given in pseudocode below.

Brute Force Factoring
<pre><code>def factorize(n):
    # n is a number, return the smallest factor of n
    for i from 2 to n-1:
        if i divides n:
            return i
        return FALSE
        </code></pre>


First step: take the pseudocode of factorize and make it into a function. I modified the code to have it return a list. If the list is empty, the number is prime. Otherwise, the list will contain all factors of the number.

In [29]:
def factorize(n):
    factors=[]
    for i in range(2, n-1):
        if n % i == 0:
            factors.append(i)
            i+=1
    return factors

Second step: test the function with two values, one that is prime and one that is not prime.

In [30]:
print(factorize(97))
print(factorize(98))

[]
[2, 7, 14, 49]


Since 97 is prime, the returned list was empty. Since 98 is not prime, the list contained all of the factors of 98. The function works!

Next, it was time to try to decrypt some messages from Piazza. First up is the message Beth posted here:https://piazza.com/class/kihq45efsn945j?cid=138_f1.

In [31]:
n = 703
e = 113

Since n and e are given, we need to find d. The first step in that process is to find p and q, which we can accomplish by using our factorize function on n.

This is also a good time to finally use the time library we imported in the very first step. The function 'time.process_time' will return the time it takes to run the current process in fractional seconds. (Details about the time library can be found here: https://docs.python.org/3/library/time.html.) We will use this function every time we decode from now on, so we can get a clear example of when brute-force factoring is no longer effective.

In [32]:
print(factorize(703))
print(time.process_time())

[19, 37]
0.870055


There are only two results, so we now know the values of p and q.

In [33]:
p, q = 19, 37

Now we have everything we need to find d using the same steps we used above. First, we need to find d using e, p, and q.

In [34]:
d = Find_Private_Key_d(e, p, q)
print(d)

281


Now that we have the private key d, we can decode the message as before. First, we set a variable equal to the encoded message, then decode it, then print the decoded message.

In [35]:
beth_message = [544, 472, 233, 242, 250, 555, 591, 550, 376, 364, 210, 165, 526, 47, 555, 95, 476, 210, 242, 165, 95, 233, 526, 210, 376, 95, 233, 242, 364, 210, 242, 224, 376, 95, 364, 233, 47, 242, 210, 555, 376, 165, 472, 242, 555, 410, 242, 250, 95, 233, 233, 47, 145]
beth_decoded = Decode(n, d, beth_message)
print(beth_decoded)

The Conquistador's treasure is buried south of Creed.


Now we can send a reply. I had no clue what this quote was from, and I said so in my reply. Encoding the reply follows the same process as all encodes above. We will again print the encoded message just to show what the message looks like after encryption.

In [36]:
russ_reply =  "I have no idea what this references but I'll take the treasure, please."
russ_encoded = Encode(n, e, russ_reply)
print(russ_encoded)

[517, 242, 472, 526, 416, 233, 242, 591, 555, 242, 364, 47, 233, 526, 242, 541, 472, 526, 165, 242, 165, 472, 364, 210, 242, 95, 233, 410, 233, 95, 233, 591, 511, 233, 210, 242, 224, 376, 165, 242, 517, 476, 90, 90, 242, 165, 526, 160, 233, 242, 165, 472, 233, 242, 165, 95, 233, 526, 210, 376, 95, 233, 157, 242, 519, 90, 233, 526, 210, 233, 145]


In order to show the difference in process time from a gradual standpoint, we will next brute-force factor an n will a slightly bigger value. (I had to use a post from Beth again because I and my classmates immediately went to very large n values!)

In [37]:
n = 130177
e = 13

Once again, we will factorize n. We will also show the process time for a larger number.

In [38]:
print(factorize(n))
print(time.process_time())

[349, 373]
0.925877657


We were still able to factorize a number with twice as many digits in not much more time. (The time taken for a three-digit number was 1.139239814 seconds. For a six digit number it was 1.182804814 seconds. Also, the times will change each time this function is run in this notebook. These were the times shown the very first time I ran the function.) Now that we have p and q, we can proceed as before: set p and q, find d, then decrypt the message.

In [39]:
p, q = 349, 373
d = Find_Private_Key_d(e, p, q)
print(d)

59749


Now that we know the value of d, we will decrypt and print the message.

In [40]:
beth_message =  [85308, 48594, 15927, 71285, 61037, 15927, 767, 23406, 71285, 23406, 48594, 12676, 37298, 100459, 71285, 90170, 61037, 38946, 38783, 23406, 71285, 90170, 71285, 113492, 38946, 38946, 86792, 15927, 90170, 37298, 71285, 12676, 767, 71285, 15927, 121493, 15927, 37298, 71285, 12676, 84628, 71285, 29228, 38946, 38783, 71285, 90170, 110238, 15927, 71285, 81798, 110238, 38946, 37298, 100459, 126494, 71285, 29228, 38946, 38783, 71285, 90170, 110238, 15927, 71285, 38946, 37298, 86792, 29228, 71285, 38946, 84628, 84628, 71285, 61037, 29228, 71285, 90170, 71285, 61037, 12676, 23406, 10510]
beth_decoded = Decode(n, d, beth_message)
print(beth_decoded)

The best thing about a Boolean is even if you are wrong, you are only off by a bit.


I love dad jokes. This would make a great dad joke. So I responded with a dad joke, computer science edition.

In [41]:
russ_reply = "Just bought a book, '1001 Amazing Stories about Binary Code.' I was disappointed to find it only had nine pages."
russ_encoded = Encode(n, e, russ_reply)
print(russ_encoded)

[86586, 38783, 767, 23406, 71285, 61037, 38946, 38783, 100459, 48594, 23406, 71285, 90170, 71285, 61037, 38946, 38946, 1607, 126494, 71285, 92171, 81670, 115640, 115640, 81670, 71285, 84148, 75888, 90170, 119480, 12676, 37298, 100459, 71285, 61795, 23406, 38946, 110238, 12676, 15927, 767, 71285, 90170, 61037, 38946, 38783, 23406, 71285, 113492, 12676, 37298, 90170, 110238, 29228, 71285, 112627, 38946, 38831, 15927, 10510, 92171, 71285, 90769, 71285, 81798, 90170, 767, 71285, 38831, 12676, 767, 90170, 114803, 114803, 38946, 12676, 37298, 23406, 15927, 38831, 71285, 23406, 38946, 71285, 84628, 12676, 37298, 38831, 71285, 12676, 23406, 71285, 38946, 37298, 86792, 29228, 71285, 48594, 90170, 38831, 71285, 37298, 12676, 37298, 15927, 71285, 114803, 90170, 100459, 15927, 767, 10510]


Factorize has proven efficient with 3 and 6-digit numbers. Now it is time to see how well it will work against very large values of n, using a code that was posted to Piazza.

In [42]:
n = 2426553226996706099
e = 5

We will not run this program in this Jupyter notebook for one reason: it takes way too long. (I let the program run for 90 minutes on my computer before stopping it, putting my laptop up on a cooling stand, and shutting it down to cool off. At one point my cpu usage for Python was listed at 24.4%, and my power consumption was rated at "very high.") But why? If we analyze one line in the factorize function, the reason becomes apparent.

for i in range(2, n-1):

This loop will run n - 2 times. (It is n - 2 times and not n - 1 times because the loop starts at the value of 2.) Using our n value from above, that means in order to find the factors of n, the loop will have to go through 2,426,553,226,996,706,097 iterations!

So back to a code that is breakable with our factorize function for my third and final example: my own!


In [43]:
n = 47448937
e = 5

Use the same methods as above to factor and show time taken to factor.

In [44]:
print(factorize(n))
print(time.process_time())

[6163, 7699]
4.085779916


Even without going to the extremes of the above 19-digit code, we can see how quickly factorize becomes unusable. Merely going from 6 digits to 8 digits took the process time from around one second (the time will vary greatly when running this notebook) to over 4 seconds! An increase of nearly 400% just by adding two digits!

Since we were able to factor with factorize, we can now find d and decode the message.

In [45]:
p, q = 6163, 7699
d = Find_Private_Key_d(e, p, q)
print(d)

37948061


With the value of d (37948061), we can decode the message.

In [46]:
russ_message = [24301984, 42671524, 46500509, 19910357, 15200115, 33554432, 23909938, 37326339, 3071463, 30986422, 23885424, 33554432, 32654648, 6208916, 37326339, 20113099, 23885424, 33554432, 30986422, 6208916, 33554432, 23885424, 19910357, 20113099, 6208916, 35723230, 23885424, 33554432, 46531597, 19910357, 35723230, 33554432, 35723230, 23885424, 20113099, 6208916, 35723230, 23885424, 33554432, 19910357, 23885424, 46531597, 37326339, 31559235, 30304999, 33554432, 23909938, 37326339, 6208916, 28080892, 23885424, 33554432, 12783961, 30304999, 33554432, 20113099, 6208916, 12783961, 19861205, 3071463, 30986422, 23885424, 37326339, 39135393, 33554432, 2070822, 19601152, 46531597, 30986422, 33554432, 44170225, 46531597, 42671524, 33554432, 30986422, 19601152, 23885424, 33554432, 23909938, 46500509, 15200115, 15200115, 23885424, 42671524, 30986422, 33554432, 12783961, 46500509, 42671524, 30986422, 46531597, 28080892, 23885424, 33554432, 30304999, 6208916, 3071463, 33554432, 12783961, 46531597, 35723230, 23885424, 33554432, 42671524, 6208916, 33554432, 32654648, 46531597, 37326339, 43457803]
print(Decode(n, d, russ_message))

Using brute force to encode and decode nearly broke my computer! What was the biggest mistake you made so far?


But this was my message! So how did I encode it? I already knew p and q since I chose them, which meant I knew n. I found my public key as we've done throughout this notebook.

In [47]:
n, e = Find_Public_Key_e(p, q)
print(n, e)

47448937 5


We already had d (which we don't need to encode the message). All that was left was to encode the message using the Encode function.

In [48]:
russ_message = "Using brute force to encode and decode nearly broke my computer! What was the biggest mistake you made so far?"
russ_encoded = (Encode(n, e, russ_message))
print(russ_encoded)

[24301984, 42671524, 46500509, 19910357, 15200115, 33554432, 23909938, 37326339, 3071463, 30986422, 23885424, 33554432, 32654648, 6208916, 37326339, 20113099, 23885424, 33554432, 30986422, 6208916, 33554432, 23885424, 19910357, 20113099, 6208916, 35723230, 23885424, 33554432, 46531597, 19910357, 35723230, 33554432, 35723230, 23885424, 20113099, 6208916, 35723230, 23885424, 33554432, 19910357, 23885424, 46531597, 37326339, 31559235, 30304999, 33554432, 23909938, 37326339, 6208916, 28080892, 23885424, 33554432, 12783961, 30304999, 33554432, 20113099, 6208916, 12783961, 19861205, 3071463, 30986422, 23885424, 37326339, 39135393, 33554432, 2070822, 19601152, 46531597, 30986422, 33554432, 44170225, 46531597, 42671524, 33554432, 30986422, 19601152, 23885424, 33554432, 23909938, 46500509, 15200115, 15200115, 23885424, 42671524, 30986422, 33554432, 12783961, 46500509, 42671524, 30986422, 46531597, 28080892, 23885424, 33554432, 30304999, 6208916, 3071463, 33554432, 12783961, 46531597, 35723230, 

The first reply was from Connor Monk, and we will decrypt and print it below.

In [49]:
connor_message =  [32767302, 33554432, 30986422, 6208916, 6208916, 28080892, 33554432, 30986422, 6208916, 6208916, 33554432, 31559235, 6208916, 19910357, 15200115, 33554432, 30986422, 6208916, 33554432, 19910357, 6208916, 30986422, 46500509, 20113099, 23885424, 33554432, 30986422, 19601152, 46531597, 30986422, 33554432, 46500509, 30986422, 33554432, 44170225, 46531597, 42671524, 33554432, 12783961, 30304999, 33554432, 19987205, 2194748, 45665365, 33554432, 32654648, 3071463, 19910357, 20113099, 30986422, 46500509, 6208916, 19910357, 33554432, 30986422, 19601152, 46531597, 30986422, 33554432, 35723230, 46500509, 35723230, 19910357, 42775262, 30986422, 33554432, 44170225, 6208916, 37326339, 28080892, 33554432, 44170225, 46500509, 30986422, 19601152, 33554432, 19910357, 23885424, 15200115, 46531597, 30986422, 46500509, 7189934, 23885424, 33554432, 35723230, 33554432, 7189934, 46531597, 31559235, 3071463, 23885424, 42671524]
print(Decode(n, d, connor_message))

I took too long to notice that it was my FME function that didn't work with negative d values


When it comes to decrypting, you can do it pretty easily using brute force up until about 9 digits. (This was also explored above in my post from the mastery workbook). After that you will need some other function to try to break the encryption and find the private key d. There were many posts on Piazza about Pollard's Rho, but that is not the subject of my extra feature, so I will not discuss it here. Needless to say you can not brute force the types of primes used in commercial encryption without waiting years, perhaps decades, for the result.


## Custom Feature ##


Not much emphasis has been placed on selecting the relatively prime numbers. This is understandable since creating a working RSA project is a daunting task in and of itself. However, now that the RSA program is working, I have decided to address that task with my custom feature. For this feature I have written a function that:

- prompts the user for a number from 3 to 10,000
- generates a list of prime numbers in the range of the number the user entered
- randomly selects two prime numbers from that list
- ensures that those numbers are relatively prime
- returns those two relatively prime numbers to the user to use as the values for p and q, and also the value of n

The first function is a Python implementation of something we did for a Mastery Workbook: the Sieve of Erastosthenes. This function will take a user input (described later), go through all values from 2 up to and including that input, and place all prime numbers in this range into a list. It does this via the Sieve: finds prime numbers, then eliminates their multiples in the range, resulting in a list of prime numbers from which p & q can be chosen.

It will then utilize the random library to randomly choose two prime numbers from that list, run them through the Euclidean_Algorithm function above to make sure they are relatively prime, and, if so, return those numbers as a tuple of the values p and q. It will also compute the value for n and return it.

In [50]:
def prime_eratosthenes(n):
    """
    takes user input for value of n in the range of 3 - 10,000
    uses Sieve of Eratosthenes to find all primes from 2 to n
    :param n: user entered number
    :return: two relatively prime numbers
    """
    prime_list = []
    composite_list = []
    for i in range(2, n+1):
        #Iterates through every number from 2 to n to see if it is prime
        if i not in composite_list:
            #checks if i is composite by looking in composite list, if not, adds to prime list
            prime_list.append(i)
            for j in range(i*i, n+1, i):
                #if i is prime, then it adds all multiples of i up to n to a list of non-primes
                composite_list.append(j)

    while True:
        x = random.choice(prime_list)           #randomly picks a number from the generated list of primes
        y = random.choice(prime_list)           #randomly picks another number from the generated list of primes
        gcd = Euclidean_Algorithm(x, y)         #uses Euclidean algorithm to make sure numbers are relatively prime
        if gcd == 1:
            return(x,y), x*y                    #returns numbers for p and q

The next function is just a tidier way of asking the user to input a number in the range of 3 to 10,000. I stopped at 10,000 because after that the program started to slow down too much. The function implemented the way it is provides the user with a very fast way of finding two random values for p and q in a very wide range, and returns those values, and n, in less than one second for the user to use in the rest of the RSA program.

In [51]:
def get_input():
    x = 0
    while x < 3 or x > 10000:                       #ensures user enters a value in appropriate range
        try:                                        #ensures user enters a number
            x = int(input("Enter a number between 3 and 10,000: "))
        except:
            pass
    (p, q), n = prime_eratosthenes(x)               #gets values of p, q, and n to return using above function
    print("(p, q) =", (p,q), "n =", n)
    print(time.process_time())                      #added to show how long the function takes when values of n increase

We are now ready to test our functions!

In [52]:
get_input()

Enter a number between 3 and 10,000:  9999


(p, q) = (7907, 7207) n = 56985749
4.957274804


That concludes the RSA project! It was a lot of work, but I have to say I am proud of what I've done. Hopefully you've enjoyed working through this notebook.

**Please use the grading guide below to self grade your Custom Feature.**

GRADING FOR CUSTOM FEATURE:

15 pts  Wow.  It is amazing.
        Shows initiative and originality. 
        You did something extra special and pushed yourself.
        You went beyond all expectations 
        You broke the rules in a creative way. 
        Your coding and commenting is exceptional.
        Excellent 

10 - 14 points - It is GOOD! 
        You were a “self starter”. 
         You did everything  requested. 
        All expectations met. 
       	You did a very good job. 
        Good use of commenting.
        Shows mastery of skills.

5 -10 points  - Okay.
        Minimum requirements.
        Commenting weak.

0 -5 points - It is not finished. 
          Does meet objectives. 
            Did not follow directions. 
            Choose not to do this part (which is a totally fine choice)


## Self grading of custom feature ##

This is no time to be modest. For my custom feature I'd give myself the full 15 points. I am a first-semester computer science student, and in three weeks I’ve coded a fully functioning RSA program without using any custom cryptography library. To me that is 100-point worthy in and of itself. But the project demanded more, so I did more.

I thought of just doing a main function, but to me that would be writing code to use code that I already wrote. I really wanted to do something related to why the most common e is 65,537, but that would have been a research paper and not a coding project. I wanted to add one more thing to make the process simpler. I got my idea when it came to the code breaking portion of the project.

For my coded message in the code breaking section, I forgot what I had chosen for p and q, so I had to factor n and find p and q again. That’s when the idea came to me. I remembered I had chosen this p and q by finding a list of prime numbers on Wikipedia, then picking two four-digit numbers to use as my values. So why not take the extra portion to automate the picking of these numbers?

Coding the Sieve of Erastosthenes was not easy. I had a general idea but kept messing up the implementation. It was when I finally added two lists to my function that I could manipulate that I got it to work. I chose values in the range of 3 – 10,000 so the function would always be able to return a tuple of relative primes, and so it would compute them rather quickly. (Up to 100,000 did not take terribly long, under one minute, but it still took much longer than 10,000.) Having two four-digit values also allowed for very big potential values for n, so I think these extra functions would provide value to the user looking to encrypt. While the input function doesn’t do much mathematically, I think it makes for a better-looking end product.

I pushed myself because I wanted to stop after the RSA. My coding experience prior to this class was one semester of C++ 15 years ago. My last math class was in 1998. The RSA was challenging enough, but I wanted the opportunity for full credit, so I made the extra functions using material that was covered in class that I think makes for a better final product.

All of this is why I think I deserve the full 15 points. I made an RSA program that works, and then I made that program better by helping the user chose the initial values that set everything in motion.